In [50]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import CRUD Python module file and class name
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "Orbit911!"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Load Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Page layout: logo, title, filter controls, table, charts/map.
app.layout = html.Div([
    html.Center(
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
        style={'height': '130px', 'marginTop': '6px'} # Adjusts height of logo
                )
    ),
        html.Center(html.B(html.H1('CS-340 Dashboard - Nate Ryals'))),
    html.Hr(),
    
    # Radio buttons to switch rescue filters
    html.Div([
        html.Label('Rescue Filter'),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',
            inline=True
        ),
    ]),
    
    html.Hr(),    
        
    # Interactive table
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         filter_action = "native",
                         sort_action = "native",
                         sort_mode = "multi",
                         row_selectable = 'single',
                         selected_rows = [0],
                         style_table={'overflowX': 'auto'},
                         style_header={'fontWeight': 'bold'},
                         page_current = 0,
                         page_size = 10,
                        ),
    html.Br(),
    html.Hr(),
        
# Sets up the dashboard so that pie chart and geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
# Add code to filter interactive data table with MongoDB queries
    
    # Base (no filter)
    query = {}
    AGE_WEEKS_MAX = 104
    
    # Case-insensitive regex for breed names
    def rx(breed):
        return {"breed": {"$regex": breed, "$options": "i"}}
    
    if filter_type == 'Water Rescue':
        query = {
            "$and": [
                {"age_upon_outcome_in_weeks": {"$lte": AGE_WEEKS_MAX}},
                {"$or": [rx("Labrador Retriever"),
                         rx("Chesapeake Bay Retriever"),
                         rx("Newfoundland")]}
            ]
        }
    
    elif filter_type == 'Mountain or Wilderness Rescue':
        query = {
            "$and": [
                {"age_upon_outcome_in_weeks": {"$lte": AGE_WEEKS_MAX}},
                {"$or": [rx("German Shepherd"),
                         rx("Alaskan Malamute"),
                         rx("Siberian Husky"),
                         rx("Border Collie"),
                         rx("Bloodhound")]}
            ]
        }
    
    elif filter_type == 'Disaster or Individual Tracking':
        query = {
            "$and": [
                {"age_upon_outcome_in_weeks": {"$lte": AGE_WEEKS_MAX}},
                {"$or": [rx("German Shepherd"),
                         rx("Doberman Pinscher"),
                         rx("Golden Retriever"),
                         rx("Bloodhound"),
                         rx("Rottweiler")]}
            ]
        }
    
    else:
        query = {} # Reset

    # Ensures a valid table is always returned
    results = db.read(query) or []
    dfx = pd.DataFrame.from_records(results)
    
    # Drop _id to avoid ObjectId errors in table
    if '_id' in dfx.columns:
        dfx.drop(columns=['_id'], inplace=True, errors='ignore')
    
    # Prevents table from crashing if empty
    if dfx.empty:
        return df.to_dict('records')
    
    return dfx.to_dict('records')
    

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    # Falls back to default if nothing rendered
    dfx = pd.DataFrame.from_dict(viewData) if viewData is not None else df.copy()
    if dfx.empty:
        return html.Div("No data to chart.", style={'opacity': 0.7})

    # Use 'breed' when available; else uses the first column
    col = 'breed' if 'breed' in dfx.columns else dfx.columns[0]
    counts = dfx[col].fillna('Unknown').value_counts().head(10).reset_index()
    counts.columns = [col, 'count']

    fig = px.pie(counts, names=col, values='count', title='Top Breeds by Count')
    return [dcc.Graph(figure=fig)]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    selected_columns = selected_columns or []
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]

# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server() 

Dash app running on https://rogersimon-buffalogroup-3000.codio.io/proxy/8050/
